In [2]:
# 1. Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

In [3]:
# 2. Load Data
df = pd.read_csv(r"E:\Python\Research\NextEnergy\XAI-ML-SOC-Prediction-TimeSeries\Dataset\Outcome\all_data.csv")


In [5]:
df.head()

,Time,Voltage,Current,Temperature,SOC
0,0.0,4.175155,-3.691549e-06,15.96381,1.0
1,0.1,4.175155,-9.512327e-07,15.96381,1.0
2,0.2,4.174967,-4.821614e-07,15.96381,1.0
3,0.3,4.175153,-2.413463e-07,15.96381,1.0
4,0.4,4.174966,-8.059866e-08,15.96381,1.0


In [7]:

# 3. Feature Engineering: Add Power
df['Power'] = df['Voltage'] * df['Current']

# 4. Normalize features
features = ['Voltage', 'Current', 'Temperature', 'Power']
target = 'SOC'

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(df[features])
y_scaled = scaler_y.fit_transform(df[[target]])


In [3]:
# 5. Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)


In [8]:

class SequenceGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, window_size=100, batch_size=64):
        self.X = X
        self.y = y
        self.window_size = window_size
        self.batch_size = batch_size

    def __len__(self):
        return (len(self.X) - self.window_size) // self.batch_size

    def __getitem__(self, idx):
        Xs, ys = [], []
        start_idx = idx * self.batch_size
        for i in range(start_idx, start_idx + self.batch_size):
            if i + self.window_size < len(self.X):
                Xs.append(self.X[i:i+self.window_size])
                ys.append(self.y[i+self.window_size])
        return np.array(Xs), np.array(ys)

In [9]:
# 7. Instantiate Generators
window_size = 100
batch_size = 64
train_gen = SequenceGenerator(X_train, y_train, window_size, batch_size)
val_gen = SequenceGenerator(X_val, y_val, window_size, batch_size)

In [10]:
# 8. Build LSTM Model
model = Sequential([
    LSTM(64, input_shape=(window_size, len(features))),
    Dense(32, activation='relu'),
    Dense(1)
])

In [11]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                17664     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 19777 (77.25 KB)
Trainable params: 19777 (77.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 9. Train Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=1,
    verbose=1
)




 77175/113089 [===================>..........] - ETA: 26:45 - loss: 8.6973e-04 - mae: 0.0212

In [ ]:

# 10. Plot Training History
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel("Epochs")
plt.ylabel("Loss / MAE")
plt.title("LSTM Training Metrics")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 11. Evaluate on Validation Set
X_val_batch, y_val_batch = val_gen[0]
y_pred_scaled = model.predict(X_val_batch)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_val_batch)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"\n📊 Evaluation Metrics:\nMAE:  {mae:.5f}\nRMSE: {rmse:.5f}\nR²:   {r2:.5f}")


In [ ]:
# 12. Plot Prediction vs. Ground Truth
plt.figure(figsize=(12, 5))
plt.plot(y_true[:500], label='True SOC', linewidth=2)
plt.plot(y_pred[:500], label='Predicted SOC', linewidth=2)
plt.title('Predicted vs. True SOC (First 500 Samples)')
plt.xlabel("Time Step")
plt.ylabel("SOC")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# 13. Save Model
model.save("lstm_soc_model.h5")
print("\n✅ Model saved as 'lstm_soc_model.h5'")